In [1]:
import os
from osl import source_recon, utils
from dask.distributed import Client

subject_names = ["bm_014", 
                 "ca_001", 
                 "ca_019",
                 "cc_007",
                 "cm_013",
                 "co_006",
                 "dm_022",
                 "ds_021",
                 "el_018",
                 "gb_020",
                 "gh_017",
                 "gp_011",
                 "gv_005",
                 "lf_012",
                 "lr_008",
                 "mp_004",
                 "pe_009",
                 "pl_016",
                 "pr_015",
                 "ra_003",
                 "re_002",
                 "sg_010"] 

# Sort by increasing number id
sorted_subject_names = sorted(subject_names, key=lambda x: int(''.join(filter(str.isdigit, x))))

BASE_DIR = "/run/user/1001/gvfs/smb-share:server=sum-lpnc-nas.u-ga.fr,share=securevault/LPNC-SecureVault/MEGAGING"
# OUTPUT_DIR = f"{BASE_DIR}/Processed/osl_processing/derivatives/coreg"
OUTPUT_DIR = "/home/clement/Bureau/derivatives"

PREPROC_DIR = f"{BASE_DIR}/Processed/osl_processing/Mot_Generation/concatenated_data"
ANAT_DIR = "/media/clement/Transcend_Clement2/Research_Projects/MEGAGING/static_analysis/data/MRI"
FSL_DIR = "/home/clement/Bureau/fsl"

In [ ]:
import numpy as np

def fix_headshape_points(outdir, subject, preproc_file, smri_file, epoch_file):
    # Load saved headshape and nasion files
    filenames = source_recon.rhino.get_coreg_filenames(outdir, subject)
    hs = np.loadtxt(filenames["polhemus_headshape_file"])
    nas = np.loadtxt(filenames["polhemus_nasion_file"])
    lpa = np.loadtxt(filenames["polhemus_lpa_file"])
    rpa = np.loadtxt(filenames["polhemus_rpa_file"])

    # Remove headshape points on the nose
    remove = np.logical_and(hs[1] > max(lpa[1], rpa[1]), hs[2] < nas[2])
    hs = hs[:, ~remove]

    # Overwrite headshape file
    utils.logger.log_or_print(f"overwritting {filenames['polhemus_headshape_file']}")
    np.savetxt(filenames["polhemus_headshape_file"], hs)

In [ ]:
if __name__ == "__main__":
    utils.logger.set_up(level="INFO")
    source_recon.setup_fsl(FSL_DIR)

    config = """
        source_recon:
        - extract_fiducials_from_fif: {}
        - fix_headshape_points: {}
        - compute_surfaces:
            include_nose: False
        - coregister:
            use_nose: False
            use_headshape: True
            n_init: 50
    """

    # Setup parallel workers
    client = Client(n_workers=20, threads_per_worker=1)

    # Get input files
    subjects = []
    preproc_files = []
    smri_files = []

    for subject in sorted_subject_names:
        subjects.append(f"{subject}")

        smri_file_path = os.path.join(ANAT_DIR, f"aging_{subject}.nii")
        smri_files.append(smri_file_path)

        preproc_file = f"{subject}_concatenated_raw.fif"
        preproc_file_path = os.path.join(PREPROC_DIR, preproc_file)
        preproc_files.append(preproc_file_path)
            
            
    # Run batch source reconstruction
    source_recon.run_src_batch(
        config,
        src_dir=OUTPUT_DIR,
        subjects=subjects,
        preproc_files=preproc_files,
        smri_files=smri_files,
        extra_funcs=[fix_headshape_points],
        dask_client=True,
    )

In [ ]:
# For unconcatenated files

# for subject in sorted_subject_names:
#     smri_file_path = os.path.join(ANAT_DIR, f"aging_{subject}.nii")
    
#     PREPROC_PATH = f"{BASE_DIR}/Processed/osl_processing/Mot_Generation/{subject}/"
    
#     for trial_num in range(1, ntrials + 1):  # 20 trials per run
#         file_name = f"trial{trial_num:03d}_raw.fif"
#         preproc_file_path = os.path.join(PREPROC_PATH, file_name)

#         if os.path.exists(preproc_file_path):  # Check if the file exists
#             label, _ = os.path.splitext(file_name)
#             subjects.append(f"{label}_{subject}")
#             preproc_files.append(preproc_file_path)
#             smri_files.append(smri_file_path)

#         for additional_trial_num in range(2, nruns + 1):  # 4 runs
#             file_name = f"trial{trial_num:03d}_{additional_trial_num:02d}_raw.fif"
#             preproc_file_path = os.path.join(PREPROC_PATH, file_name)

#             if os.path.exists(preproc_file_path):  # Check if the file exists
#                 label, _ = os.path.splitext(file_name)
#                 subjects.append(f"{label}_{subject}")
#                 preproc_files.append(preproc_file_path)
#                 smri_files.append(smri_file_path)

SOURCE RECON

In [2]:
# These subjects have bad segmentation
# subject_names = ["co_006",
#                  "ds_021",
#                  "mp_004",]


subject_names = ["bm_014", 
                 "ca_001", 
                 "ca_019",
                 "cc_007",
                 "cm_013",
                 "dm_022",
                 "el_018",
                 "gb_020",
                 "gh_017",
                 "gp_011",
                 "gv_005",
                 "lf_012",
                 "lr_008",
                 "pe_009",
                 "pl_016",
                 "pr_015",
                 "ra_003",
                 "re_002",
                 "sg_010"] 

# Sort by increasing number id
sorted_subject_names = sorted(subject_names, key=lambda x: int(''.join(filter(str.isdigit, x))))

BASE_DIR = "/run/user/1001/gvfs/smb-share:server=sum-lpnc-nas.u-ga.fr,share=securevault/LPNC-SecureVault/MEGAGING"
SRC_DIR = "/home/clement/Bureau/derivatives"

In [3]:
config = """
    source_recon:
    - forward_model:
        model: Single Layer
    - beamform_and_parcellate:
        freq_range: [1, 45]
        chantypes: [mag, grad]
        rank: {meg: 60}
        parcellation_file: Glasser52_binary_space-MNI152NLin6_res-8x8x8.nii.gz
        method: spatial_basis
        orthogonalisation: symmetric
"""

if __name__ == "__main__":
    utils.logger.set_up(level="INFO")
    source_recon.setup_fsl(FSL_DIR)

    # Get input files
    subjects = []
    preproc_files = []
    smri_files = []

    for subject in sorted_subject_names:
        subjects.append(f"{subject}")

        smri_file_path = os.path.join(ANAT_DIR, f"aging_{subject}.nii")
        smri_files.append(smri_file_path)

        preproc_file = f"{subject}_concatenated_raw.fif"
        preproc_file_path = os.path.join(PREPROC_DIR, preproc_file)
        preproc_files.append(preproc_file_path)

    
    # Setup parallel processing
    client = Client(n_workers=16, threads_per_worker=1)

    # Run beamforming and parcellation
    source_recon.run_src_batch(
        config,
        src_dir=SRC_DIR,
        subjects=subjects,
        preproc_files=preproc_files,
        smri_files=smri_files,
        dask_client=False,
    )

 OSL osl_logger: handler 'console' level set to 'INFO'
 OSL Logger Started


/home/clement/Bureau/fsl/envs/osl/lib/python3.8/site-packages/distributed/dashboard/core.py:20: UserWarning: 
Dask needs bokeh >= 2.4.2, < 3 for the dashboard.
You have bokeh==3.1.1.
Continuing without the dashboard.
  warnings.warn(
INFO:distributed.http.proxy:To route to workers diagnostics web server please install jupyter-server-proxy: python -m pip install jupyter-server-proxy
/home/clement/Bureau/fsl/envs/osl/lib/python3.8/site-packages/distributed/node.py:182: UserWarning: Port 8787 is already in use.
Perhaps you already have a cluster running?
Hosting the HTTP server on port 42903 instead
  warnings.warn(


 OSL osl_logger: handler 'console' level set to 'INFO'
 logging to file: /home/clement/Bureau/derivatives/logs/osl_batch.log
 Starting OSL Batch Source Reconstruction
 Running config
 {'source_recon': [{'forward_model': {'model': 'Single Layer'}},
                  {'beamform_and_parcellate': {'chantypes': ['mag', 'grad'],
                                               'freq_range': [1, 45],
                                               'method': 'spatial_basis',
                                               'orthogonalisation': 'symmetric',
                                               'parcellation_file': 'Glasser52_binary_space-MNI152NLin6_res-8x8x8.nii.gz',
                                               'rank': {'meg': 60}}}]}
ca_001_concatenated_raw : OSL osl_logger: handler 'console' level set to 'INFO'
ca_001_concatenated_raw : logging to file: /home/clement/Bureau/derivatives/logs/ca_001_concatenated_raw_src.log
ca_001_concatenated_raw : 2024-09-26 16:31:36 : Starting OSL Pr

/home/clement/Bureau/fsl/envs/osl/lib/python3.8/site-packages/osl/source_recon/batch.py:147: RuntimeWarning: Log entries will be appended to the file. Use overwrite=False to avoid this message in the future.
  mne.utils._logging.set_log_file(logfile)


ca_001_concatenated_raw : *** OSL RHINO FORWARD MODEL COMPLETE ***
ca_001_concatenated_raw : beamform_and_parcellate
ca_001_concatenated_raw : bandpass filtering: 1-45 Hz
ca_001_concatenated_raw : beamforming.make_lcmv
ca_001_concatenated_raw : chantypes: ['mag', 'grad']
ca_001_concatenated_raw : rank: {'meg': 60}
ca_001_concatenated_raw : *** RUNNING OSL MAKE LCMV ***
ca_001_concatenated_raw : variance for chantype mag is 2.4136594046292497e-26
ca_001_concatenated_raw : variance for chantype grad is 1.0780300982318391e-23
ca_001_concatenated_raw : saving /home/clement/Bureau/derivatives/ca_001/beamform/filters-lcmv.h5
ca_001_concatenated_raw : *** OSL MAKE LCMV COMPLETE ***
ca_001_concatenated_raw : beamforming.apply_lcmv
ca_001_concatenated_raw : spatial_resolution = 8 mm
ca_001_concatenated_raw : parcellation
ca_001_concatenated_raw : using file Glasser52_binary_space-MNI152NLin6_res-8x8x8.nii.gz
ca_001_concatenated_raw : gridstep = 8 mm


...

gb_020_concatenated_raw : *** OSL RHINO FORWARD MODEL COMPLETE ***
gb_020_concatenated_raw : beamform_and_parcellate
gb_020_concatenated_raw : bandpass filtering: 1-45 Hz
gb_020_concatenated_raw : beamforming.make_lcmv
gb_020_concatenated_raw : chantypes: ['mag', 'grad']
gb_020_concatenated_raw : rank: {'meg': 60}
gb_020_concatenated_raw : *** RUNNING OSL MAKE LCMV ***
gb_020_concatenated_raw : variance for chantype mag is 6.598229144364775e-26
gb_020_concatenated_raw : variance for chantype grad is 3.2569081122968066e-23
gb_020_concatenated_raw : saving /home/clement/Bureau/derivatives/gb_020/beamform/filters-lcmv.h5
gb_020_concatenated_raw : *** OSL MAKE LCMV COMPLETE ***
gb_020_concatenated_raw : beamforming.apply_lcmv
gb_020_concatenated_raw : spatial_resolution = 8 mm
gb_020_concatenated_raw : parcellation
gb_020_concatenated_raw : using file Glasser52_binary_space-MNI152NLin6_res-8x8x8.nii.gz
gb_020_concatenated_raw : gridstep = 8 mm


gb_020_concatenated_raw : symmetric orthogonalisation
gb_020_concatenated_raw : saving /home/clement/Bureau/derivatives/gb_020/parc/parc-raw.fif
gb_020_concatenated_raw : saving /home/clement/Bureau/derivatives/gb_020/parc/psd.png
gb_020_concatenated_raw : saving /home/clement/Bureau/derivatives/gb_020/parc/corr.png
dm_022_concatenated_raw : OSL osl_logger: handler 'console' level set to 'INFO'
dm_022_concatenated_raw : logging to file: /home/clement/Bureau/derivatives/logs/dm_022_concatenated_raw_src.log
dm_022_concatenated_raw : 2024-09-26 16:58:29 : Starting OSL Processing
dm_022_concatenated_raw : input : /home/clement/Bureau/derivatives/dm_022
dm_022_concatenated_raw : *** RUNNING OSL RHINO FORWARD MODEL ***
dm_022_concatenated_raw : Using bet_inskull_surf_file for single shell surface


/home/clement/Bureau/fsl/envs/osl/lib/python3.8/site-packages/osl/source_recon/batch.py:147: RuntimeWarning: Log entries will be appended to the file. Use overwrite=False to avoid this message in the future.
  mne.utils._logging.set_log_file(logfile)


dm_022_concatenated_raw : *** OSL RHINO FORWARD MODEL COMPLETE ***
dm_022_concatenated_raw : beamform_and_parcellate
dm_022_concatenated_raw : bandpass filtering: 1-45 Hz
dm_022_concatenated_raw : beamforming.make_lcmv
dm_022_concatenated_raw : chantypes: ['mag', 'grad']
dm_022_concatenated_raw : rank: {'meg': 60}
dm_022_concatenated_raw : *** RUNNING OSL MAKE LCMV ***
dm_022_concatenated_raw : variance for chantype mag is 7.023999635639898e-26
dm_022_concatenated_raw : variance for chantype grad is 3.4639931396350356e-23
dm_022_concatenated_raw : saving /home/clement/Bureau/derivatives/dm_022/beamform/filters-lcmv.h5
dm_022_concatenated_raw : *** OSL MAKE LCMV COMPLETE ***
dm_022_concatenated_raw : beamforming.apply_lcmv
dm_022_concatenated_raw : spatial_resolution = 8 mm
dm_022_concatenated_raw : parcellation
dm_022_concatenated_raw : using file Glasser52_binary_space-MNI152NLin6_res-8x8x8.nii.gz
dm_022_concatenated_raw : gridstep = 8 mm


dm_022_concatenated_raw : symmetric orthogonalisation
dm_022_concatenated_raw : saving /home/clement/Bureau/derivatives/dm_022/parc/parc-raw.fif
dm_022_concatenated_raw : saving /home/clement/Bureau/derivatives/dm_022/parc/psd.png
dm_022_concatenated_raw : saving /home/clement/Bureau/derivatives/dm_022/parc/corr.png
 OSL osl_logger: handler 'console' level set to 'INFO'
 logging to file: /home/clement/Bureau/derivatives/logs/osl_batch.log
 Processed 19/19 files successfully
 *********************************************************************
 * REMEMBER TO CHECK REPORT: /home/clement/Bureau/derivatives/report *
 *********************************************************************
